In [1]:
# Legal Reasoning Project, NCCU (2025)
# 2_kg_eval.ipynb: Refine a knowledge graph by merging semantically equivalent or highly similar nodes and creating similarity edges between related nodes, thereby optimizing the graph structure and addressing sparsity issues.

## Objective (problems solved by this ipynb file):

This file marks clues of flaws of the knowledge graph for better evaluation.

## Section 1: Obversation

In [2]:
import json
import networkx as nx
from collections import Counter

In [3]:
# manually upload knowledge_graph.json

In [4]:
# 讀取你的檔案
with open('knowledge_graph.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [5]:
# 1. 檢查節點類型的分佈
node_types = []
for n in data['nodes']:
    # 假設你的節點有 'node_type' 欄位，如果沒有，請根據 id 前綴判斷
    n_type = n.get('node_type', 'Unknown')
    node_types.append(n_type)

print("📊 節點類型分佈 (Node Distribution):")
print(Counter(node_types))

📊 節點類型分佈 (Node Distribution):
Counter({'Violation': 1021, 'Incident': 431, 'Cause_Direct': 421, 'Regulation': 366, 'Industry': 174, 'Medium_Specific': 52, 'Medium_Normal': 18, 'IncidentType': 17, 'Medium_General': 8, 'Cause_Basic': 4, 'Cause_Indirect': 2})


In [6]:
# 2. 檢查關係類型的分佈
edge_types = []
links = data.get('links', data.get('edges', []))
for l in links:
    # 假設邊有 'relation' 欄位
    r_type = l.get('relation', 'Unknown')
    edge_types.append(r_type)

print("\n🔗 關係類型分佈 (Relation Distribution):")
print(Counter(edge_types))


🔗 關係類型分佈 (Relation Distribution):
Counter({'VIOLATES_LAW': 19830, 'LEADS_TO': 2151, 'IS_SUBCLASS_OF': 857, 'INVOLVES_OBJECT': 432, 'OCCURS_IN': 432, 'HAS_INCIDENT_TYPE': 432, 'HAS_CAUSE': 432, 'ENABLED_BY': 4})


In [7]:
# 3. 檢查孤立節點 (Isolated Nodes)
# 如果有很多節點沒有邊，KGE 學不到東西
G = nx.node_link_graph(data)
isolated_count = list(nx.isolates(G))
print(f"\n⚠️ 孤立節點數量: {len(isolated_count)}")
if len(isolated_count) > 0:
    print(f"   範例: {isolated_count[:5]}")


⚠️ 孤立節點數量: 3
   範例: ['REG_c70ba705de9b', 'REG_f31033e6198f', 'REG_94b4e1cf8334']


## Section 2: Refine sparsity and connectivity

### **對關鍵問題一：圖譜的稀疏性與連通性 Sparsity & Connectivity**
* 問題再定義（Restatement）： 這不只是稀疏（Sparse），這是離散（Disconnected）。目前的圖譜是由 431 個近乎獨立的「小樹叢」組成的，彼此之間缺乏共享節點。
  * 現象：由於 Cause_Indirect 和 Cause_Basic 的節點數量趨近於零，這些原本應該作為「樞紐（Hubs）」將不同案件串聯起來的節點失效了。
  * 後果：GNN（如 R-GCN）的核心優勢是 Message Passing（訊息傳遞）。如果圖不連通，GNN 就退化成了一個普通的特徵編碼器（Encoder），無法從「A案件的判決」學到「對B案件的推論」，因為 A 和 B 之間沒有路徑。
* 潛在解決方案 (Solution)：
  * 強制節點共享 (Node Canonicalization)：在構建圖譜時，必須引入實體對齊機制。例如，所有提到「未戴安全帽」的描述，都應該指向同一個 node_id。
  * 虛擬節點 (Super Nodes)：人為創建幾個高層級的「概念節點」（如：墜落事故類、感電事故類），強行將同類案件連接起來，讓 Message Passing 有路可走。
  * 修復萃取邏輯：回頭檢查 Data Pipeline，為什麼 431 個案件只產生了 4 個基本原因？必須讓每個案件都至少生成出對應的 Cause_Indirect 和 Cause_Basic 節點。

In [8]:
import json
import networkx as nx
import numpy as np
from sentence_transformers import SentenceTransformer, util
from collections import defaultdict
from tqdm import tqdm

In [9]:
class OSHGraphRefiner:
    def __init__(self, json_path, model_name='paraphrase-multilingual-MiniLM-L12-v2'):
        """
        初始化圖譜優化器
        :param json_path: 原始 knowledge_graph.json 路徑
        :param model_name: 支援中文的 Sentence-BERT 模型
        """
        self.json_path = json_path
        self.model = SentenceTransformer(model_name)
        self.graph_data = self._load_data()
        self.G = self._build_initial_graph()
        print(f"✅ 初始圖譜載入完成: {self.G.number_of_nodes()} 節點, {self.G.number_of_edges()} 邊")

    def _load_data(self):
        with open(self.json_path, 'r', encoding='utf-8') as f:
            return json.load(f)

    def _build_initial_graph(self):
        G = nx.MultiDiGraph()

        # --- 1. 建立節點 (Nodes) ---
        # 增加防禦性程式設計，確保 'nodes' 鍵存在
        if 'nodes' not in self.graph_data:
            raise ValueError("❌ JSON 檔案格式錯誤：找不到 'nodes' 鍵值。")

        for node in self.graph_data['nodes']:
            # 使用 label 或 full_text 作為語意嵌入的基礎
            # 建議：為了增強推理能力，這裡可以考慮同時結合 label 和 full_text，而不僅是擇一
            text = node.get('label', '')
            if node.get('full_text'):
                text += " " + node.get('full_text')

            G.add_node(node['id'], **node, embedding_text=text)

        # --- 2. 建立邊 (Edges) ---
        # 自動偵測邊的鍵值名稱 ('links' 或 'edges')
        edge_key = None
        if 'links' in self.graph_data:
            edge_key = 'links'
        elif 'edges' in self.graph_data:
            edge_key = 'edges'
        else:
            # 如果都找不到，列出所有鍵值以供除錯
            raise KeyError(f"❌ JSON 檔案格式錯誤：找不到邊的定義 (expected 'links' or 'edges')。現有鍵值: {list(self.graph_data.keys())}")

        print(f"ℹ️ 偵測到邊的鍵值名稱為: '{edge_key}'")

        for link in self.graph_data[edge_key]:
            G.add_edge(link['source'], link['target'],
                       relation=link['relation'],
                       weight=link.get('weight', 1.0))

        return G

    def compute_embeddings(self, node_ids):
        """為指定的節點列表計算 Embedding"""
        texts = [self.G.nodes[nid].get('embedding_text', '') for nid in node_ids]
        embeddings = self.model.encode(texts, convert_to_tensor=True, show_progress_bar=True)
        return embeddings

    def merge_semantically_equivalent_nodes(self, node_type, threshold=0.90):
        """
        【解決方案 1：節點規範化】
        找出語意極度相似 (Sim > 0.9) 的同類型節點進行合併。
        這能解決「未戴安全帽」與「沒戴安全帽」被視為不同節點的問題。
        """
        print(f"\n🔍 正在掃描並合併 {node_type} 類型的重複節點...")

        # 篩選特定類型的節點
        target_nodes = [n for n, attr in self.G.nodes(data=True) if attr.get('node_type') == node_type]
        if len(target_nodes) < 2:
            return

        embeddings = self.compute_embeddings(target_nodes)

        # 計算相似度矩陣
        cosine_scores = util.cos_sim(embeddings, embeddings)

        # 建立合併映射表 (Old ID -> New Canonical ID)
        merge_map = {}
        kept_nodes = set()

        # 貪婪算法進行分群
        visited = set()
        for i in range(len(target_nodes)):
            if i in visited: continue

            current_id = target_nodes[i]
            kept_nodes.add(current_id)
            visited.add(i)

            # 找出所有與當前節點相似度高於門檻的節點
            for j in range(i + 1, len(target_nodes)):
                if j in visited: continue

                if cosine_scores[i][j] >= threshold:
                    target_id = target_nodes[j]
                    merge_map[target_id] = current_id # 將 j 合併到 i
                    visited.add(j)
                    # print(f"  🔗 合併: {self.G.nodes[target_id]['label'][:20]}... -> {self.G.nodes[current_id]['label'][:20]}...")

        # 執行圖譜重構 (Rewiring)
        self._apply_node_merging(merge_map)
        print(f"✨ 合併完成: 減少了 {len(merge_map)} 個重複節點")

    def _apply_node_merging(self, merge_map):
        """執行 NetworkX 的節點合併操作"""
        for old_id, new_id in merge_map.items():
            if old_id not in self.G: continue

            # 轉移入邊 (In-edges)
            for u, v, key, data in list(self.G.in_edges(old_id, keys=True, data=True)):
                self.G.add_edge(u, new_id, **data)

            # 轉移出邊 (Out-edges)
            for u, v, key, data in list(self.G.out_edges(old_id, keys=True, data=True)):
                self.G.add_edge(new_id, v, **data)

            # 移除舊節點
            self.G.remove_node(old_id)

    def add_similarity_edges(self, node_type, threshold=0.85, top_k=3):
        """
        【解決方案 2：建立相似邊 (Densification)】
        為相似但不完全相同的節點建立 `IS_SIMILAR_TO` 邊。
        這讓 GNN 可以跨案件傳遞訊息。
        """
        print(f"\n🕸️ 正在為 {node_type} 建立相似度連接...")
        target_nodes = [n for n, attr in self.G.nodes(data=True) if attr.get('node_type') == node_type]
        if len(target_nodes) < 2: return

        embeddings = self.compute_embeddings(target_nodes)
        cosine_scores = util.cos_sim(embeddings, embeddings)

        added_edges = 0
        # 轉為 Numpy 方便處理
        score_matrix = cosine_scores.cpu().numpy()
        np.fill_diagonal(score_matrix, 0) # 排除自己連自己

        for i in range(len(target_nodes)):
            # 找出最相似的 Top-K
            top_indices = np.argsort(score_matrix[i])[-top_k:]

            source_id = target_nodes[i]
            for idx in top_indices:
                score = score_matrix[i][idx]
                if score > threshold:
                    target_id = target_nodes[idx]

                    # 避免重複添加雙向邊
                    if not self.G.has_edge(source_id, target_id):
                        self.G.add_edge(source_id, target_id,
                                        relation='IS_SIMILAR_TO',
                                        weight=float(score))
                        added_edges += 1

        print(f"🚀 新增了 {added_edges} 條 '{node_type}' 之間的相似邊")

    def export_graph(self, output_path):
        data = nx.node_link_data(self.G)
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"💾 優化後的圖譜已儲存至: {output_path}")

In [10]:
if __name__ == "__main__":
    # 請將此路徑替換為您的檔案真實路徑
    INPUT_FILE = "knowledge_graph.json"
    OUTPUT_FILE = "knowledge_graph_connected.json"

    refiner = OSHGraphRefiner(INPUT_FILE)

    # 步驟 1: 合併重複的違規行為 (Violation)
    # 很多違規行為描述略有不同但法理意義相同，這一步可以大幅降低稀疏度
    refiner.merge_semantically_equivalent_nodes(node_type="Violation", threshold=0.92)

    # 步驟 2: 合併重複的直接原因 (Cause_Direct)
    # 例如 "墜落" 和 "從高處掉落"
    refiner.merge_semantically_equivalent_nodes(node_type="Cause_Direct", threshold=0.90)

    # 步驟 3: 建立案件間的相似性 (Incident)
    # 這一步最關鍵！它將原本孤立的 431 個案件，透過相似度連成一個「小世界網路 (Small World Network)」
    refiner.add_similarity_edges(node_type="Incident", threshold=0.85, top_k=5)

    # 步驟 4: 輸出結果
    refiner.export_graph(OUTPUT_FILE)

ℹ️ 偵測到邊的鍵值名稱為: 'edges'
✅ 初始圖譜載入完成: 2514 節點, 24570 邊

🔍 正在掃描並合併 Violation 類型的重複節點...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

✨ 合併完成: 減少了 427 個重複節點

🔍 正在掃描並合併 Cause_Direct 類型的重複節點...


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

✨ 合併完成: 減少了 90 個重複節點

🕸️ 正在為 Incident 建立相似度連接...


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

🚀 新增了 36 條 'Incident' 之間的相似邊
💾 優化後的圖譜已儲存至: knowledge_graph_connected.json
